**Import libraries**

In [1]:
import holoviews as hv
from holoviews import opts
from holoviews import streams
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import geopandas as gpd
import panel as pn
import pandas as pd
import numpy as np
from cmcrameri import cm
import cmocean

pn.extension('tabulator', sizing_mode="stretch_width")
hv.extension('bokeh')

**Figure 1**

In [ ]:
# Shapefile
fig1_philippines = (
    gpd.read_file('shapefile/climatezones/climate_type.shp')
    .replace({'Type': {4.0: 'Type IV', 3.0: 'Type III', 2.0: 'Type II', 1.0: 'Type I'}})
    .iloc[::-1]
)

# Data
fig1_file_paths = ['df_precip.csv', 'df_tas.csv', 'df_tasmin.csv', 'df_tasmax.csv']
fig1_dfs = {name: pd.read_csv(f"data/{file}").drop(columns=['Unnamed: 0']).melt(
        id_vars=['Month'], var_name='Type', value_name='Value'
    )
    for name, file in zip(['precip', 'tas', 'tasmin', 'tasmax'], fig1_file_paths)
}
fig1_df_precip, fig1_df_tas, fig1_df_tasmin, fig1_df_tasmax = fig1_dfs.values()

# Colour Palette
fig1_cmap = {
    'Type I': 'red',
    'Type II': 'blue',
    'Type III': 'gold',
    'Type IV': 'limegreen'
}

# Map Plot with Stream for Click Detection
fig1_map_plot = fig1_philippines.hvplot(
    geo=True, color='Type', cmap=fig1_cmap, hover_cols=['Type'], width=500, height=900, legend='top_right'
).opts(
    opts.Polygons(line_color='black', tools=['hover', 'tap'], active_tools=['wheel_zoom'], toolbar='above')
)

# Add Selection Stream
selection = streams.Selection1D(source=fig1_map_plot)

# Function to update plots dynamically
def update_plots(indexes):
    if not indexes:
        selected_types = ['Type I', 'Type II', 'Type III', 'Type IV']  # Default to all
    else:
        selected_types = fig1_philippines.iloc[indexes]['Type'].tolist()

    # --- Precipitation Plot ---
    precip_plots = []
    for climate_type in selected_types:
        precip_plot = fig1_df_precip[fig1_df_precip['Type'] == climate_type].hvplot.bar(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=500, height=500, alpha=1.0
        ).opts(
            opts.Bars(
                tools=['hover', 'tap'], active_tools=['wheel_zoom'], toolbar='above',
                title='Monthly Precipitation',
                ylabel='Mean Precipitation (mm)',
                axiswise=True,
            )
        )
        precip_plots.append(precip_plot)

    precip_combined = hv.Overlay(precip_plots).opts(legend_position='top', axiswise=True)

    # --- Temperature Plot ---
    temp_plots = []
    for climate_type in selected_types:
        temp_plot = fig1_df_tas[fig1_df_tas['Type'] == climate_type].hvplot.line(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=500, height=500
        ).opts(
            opts.Curve(
                tools=['hover', 'tap'], active_tools=['wheel_zoom'], toolbar='above',
                title='Monthly Temperature',
                ylabel='Temperature (°C)',
                axiswise=True
            )
        )
        tasmin_plot = fig1_df_tasmin[fig1_df_tasmin['Type'] == climate_type].hvplot.line(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=500, height=500, line_dash='dashed'
        )
        tasmax_plot = fig1_df_tasmax[fig1_df_tasmax['Type'] == climate_type].hvplot.line(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=500, height=500, line_dash='dotted'
        )
        temp_plot = temp_plot * tasmin_plot * tasmax_plot
        temp_plots.append(temp_plot)

    temp_combined = hv.Overlay(temp_plots).opts(legend_position='top', axiswise=True)

    return pn.Column(precip_combined, temp_combined)

# Bind the update function to selection
fig1_dynamic_plot = pn.bind(update_plots, selection.param.index)

# Create the Final Layout
fig1_layout = pn.Column(
    pn.Row(
        fig1_map_plot,
        fig1_dynamic_plot
    ),
    width=1980,
)

DataSourceError: shapefile/climatezones/climate_type.shp: No such file or directory

**Figure 2**

In [ ]:
# Create data for number of landfalling TCs and number of non-landfalling TCs in the Philippines from 1951 to 2013
fig2_data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    'Non-landfalling': [0.2, 0.2, 0.1, 0.2, 0.5, 0.8, 2.0, 2.0, 2.0, 1.0, 0.6, 0.5],  
    'Landfalling': [0.2, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7],      
    'total': [0.4, 0.3, 0.3, 0.5, 1.0, 1.5, 3.0, 3.0, 3.0, 2.0, 1.6, 1.2]           
}
fig2_df = pd.DataFrame(fig2_data)

# Create a stacked bar chart
fig2_plot = fig2_df.hvplot.bar(
    x='Month',
    y=['Non-landfalling', 'Landfalling'],  # Specify both columns for stacking
    stacked=True,  # Enable stacking
    color=['cornflowerblue', 'royalblue'],  # Colors for each category
    alpha=0.8,
    width=1280,
    height=720,
    legend='top_left',
).opts(
    xlabel='Month',
    ylabel='Number of TCs',
    title='Annual Seasonal Frequency of Landfalling and Non-Landfalling Tropical Cyclones\n within Philippine Area of Responsibility (PAR) Based on IBTrACS and cinco_et_al (1951-2024)',
    tools=['hover'],
    toolbar='right'
)

# Prepare data for text labels
labels_data = {
    'Month': fig2_data['Month'],  # x-axis positions (months)
    'y': [v + 0.1 for v in fig2_data['total']],  # Add an offset to move labels above the bars
    'label': [str(v) for v in fig2_data['total']]  # Text labels (total values as strings)
}
labels_df = pd.DataFrame(labels_data)

# Create labels using hv.Labels
labels = hv.Labels(labels_df, ['Month', 'y'], 'label').opts(
    text_align='center',  # Center-align the text
    text_baseline='middle',  # Vertically align the text to the middle
    text_color='black',  # Set text color to black
    text_font_size='12pt',  # Set font size for the text
)

# Overlay the bar chart and the labels
fig2_plot_with_labels = fig2_plot * labels

**Figure 4**

In [ ]:
# Read shapefile
fig4_cebu_shapefile = gpd.read_file('shapefile/phl_adm_psa_namria_20231106_shp/phl_admbnda_adm2_psa_namria_20231106.shp')
fig4_cebu = fig4_cebu_shapefile[fig4_cebu_shapefile.ADM2_EN == 'Cebu']
fig4_ncr_shapefile = gpd.read_file('shapefile/phl_adm_psa_namria_20231106_shp/phl_admbnda_adm1_psa_namria_20231106.shp')
fig4_ncr = fig4_ncr_shapefile[fig4_ncr_shapefile.ADM1_EN == 'National Capital Region (NCR)']

# Read data
fig4_precip_df = pd.read_csv('data/pr_annual.csv')
fig4_precip_cebu_df = fig4_precip_df[['Year', 'Cebu']].dropna().reset_index(drop=True)
fig4_precip_mm_df = fig4_precip_df[['Year', 'Metro Manila']].dropna().reset_index(drop=True)
fig4_tas_df = pd.read_csv('data/tm_annual.csv')
fig4_tas_cebu_df = fig4_tas_df[['Year', 'Cebu']].dropna().reset_index(drop=True)
fig4_tas_mm_df = fig4_tas_df[['Year', 'Metro Manila']].dropna().reset_index(drop=True)

fig4_cebu_map = fig4_cebu.hvplot(
    geo=True, color='gold', alpha=1.0, line_color='black', width=400, height=400
    ).opts(title='Cebu', toolbar='left', active_tools=['wheel_zoom'], tools=['hover'])
fig4_ncr_map = fig4_ncr.hvplot(
    geo=True, color='red', alpha=1.0, line_color='black', width=400, height=400
    ).opts(title='NCR (Metropolitan Manila)', toolbar='left', active_tools=['wheel_zoom'], tools=['hover'])

# Process data
el_nino = {
    1951, 1953, 1957, 1958, 1959, 1963, 1965, 1966, 1968, 1969, 1972,
    1977, 1979, 1982, 1983, 1986, 1987, 1991, 1992, 1997,
    2002, 2003, 2004, 2005, 2009, 2010, 2015, 2016, 2019
}
la_nina = {
    1954, 1955, 1956, 1964, 1970, 1971, 1973, 1976, 1984,
    1985, 1988, 1989, 1995, 1998,
    2000, 2007, 2008, 2011, 2012, 2017, 2018, 2020
}


def process_precip_data(df, location):
    df_elnino = df[df['Year'].isin(el_nino)].copy()
    df_lanina = df[df['Year'].isin(la_nina)].copy()
    df_neutral = df[~df['Year'].isin(el_nino.union(la_nina))].copy()
    
    # Use .loc to avoid SettingWithCopyWarning
    for d in [df_elnino, df_lanina, df_neutral]:
        d.loc[:, location] = d[location].round(0).astype(int)
    
    return df_elnino, df_lanina, df_neutral

cebu_precip_elnino, cebu_precip_lanina, cebu_precip_neutral = process_precip_data(fig4_precip_cebu_df, 'Cebu')
ncr_precip_elnino, ncr_precip_lanina, ncr_precip_neutral = process_precip_data(fig4_precip_mm_df, 'Metro Manila')

def fig4_plot_precipitation(df_neutral, df_elnino, df_lanina, region_name, location):
    # Plot neutral 
    neutral_plot = df_neutral.hvplot.bar(x='Year', y=location,
                                        color='silver', label='Neutral',
                                        alpha=0.8, bar_width=5,
                                        width=1000, height=500).opts(axiswise=True)
    # Plot El Niño
    elnino_plot = df_elnino.hvplot.bar(x='Year', y=location,
                                    color='orange', label='El Niño',
                                    alpha=0.8, bar_width=5,
                                    width=1000, height=500).opts(axiswise=True)
    # Plot La Niña
    lanina_plot = df_lanina.hvplot.bar(x='Year', y=location,
                                    color='cadetblue', label='La Niña',
                                    alpha=0.8, bar_width=5,
                                    width=1000, height=500).opts(axiswise=True)
    # Overlay the plots
    combined_plot = (neutral_plot * elnino_plot * lanina_plot).opts(
        title=f'Observed Annual Precipitation in {region_name}',
        xlabel='Year',
        ylabel='Precipitation (mm)',
        legend_position='top',
        tools=['hover', 'tap'],
        active_tools=['wheel_zoom'],
        toolbar='right',
        show_grid=False,
        axiswise=True
    )

    return combined_plot

fig4_cebu_precip_plot = fig4_plot_precipitation(cebu_precip_neutral, cebu_precip_elnino, cebu_precip_lanina, 'Cebu', 'Cebu')
fig4_ncr_precip_plot = fig4_plot_precipitation(ncr_precip_neutral, ncr_precip_elnino, ncr_precip_lanina, 'NCR', 'Metro Manila')

# Temperature plot
def fig4_plot_temperature(df_tas, region_name, location):
    # Plot temperature
    temp_plot = df_tas.hvplot.line(x='Year', y=location,
                                color='red',
                                width=1000, height=500).opts(
                                    title=f'Observed Annual Temperature in {region_name}',
                                    xlabel='Year',
                                    ylabel='Temperature (°C)',
                                    tools=['hover', 'tap'],
                                    active_tools=['wheel_zoom'],
                                    toolbar='right',
                                    show_grid=False,
                                    axiswise=True,
                                )
    # Add points for each year
    points = df_tas.hvplot.scatter(x='Year', y=location, color='black', size=10)

    # Overlay the plots
    combined_plot = temp_plot * points 
    return combined_plot

fig4_cebu_temp_plot = fig4_plot_temperature(fig4_tas_cebu_df,'Cebu', 'Cebu')
fig4_ncr_temp_plot = fig4_plot_temperature(fig4_tas_mm_df,'NCR', 'Metro Manila')

# Figure 4 Layout
fig4_layout_cebu = pn.Row(fig4_cebu_map, pn.Column(fig4_cebu_precip_plot, fig4_cebu_temp_plot))
fig4_layout_ncr = pn.Row(fig4_ncr_map, pn.Column(fig4_ncr_precip_plot, fig4_ncr_temp_plot))

**Figure 5**

In [ ]:
# Read data

fig5_rx1day_df = pd.read_csv('data/rx1day.csv')
fig5_rx1day_cebu_df = fig5_rx1day_df[['Year', 'Cebu']].dropna().reset_index(drop=True)
fig5_rx1day_mm_df = fig5_rx1day_df[['Year', 'Metro Manila']].dropna().reset_index(drop=True)

fig5_txx_df = pd.read_csv('data/txx.csv')
fig5_txx_cebu_df = fig5_txx_df[['Year', 'Cebu']].dropna().reset_index(drop=True)
fig5_txx_mm_df = fig5_txx_df[['Year', 'Metro Manila']].dropna().reset_index(drop=True)

# Process data
cebu_rx1day_elnino, cebu_rx1day_lanina, cebu_rx1day_neutral = process_precip_data(fig5_rx1day_cebu_df, 'Cebu')
ncr_rx1day_elnino, ncr_rx1day_lanina, ncr_rx1day_neutral = process_precip_data(fig5_rx1day_mm_df, 'Metro Manila')

# Precipitation Plot
def fig5_plot_rx1day(df_neutral, df_elnino, df_lanina, region_name, location):
    # Plot neutral 
    neutral_plot = df_neutral.hvplot.bar(x='Year', y=location,
                                        color='silver', label='Neutral',
                                        alpha=0.8, bar_width=5,
                                        width=1000, height=500).opts(axiswise=True)
    # Plot El Niño
    elnino_plot = df_elnino.hvplot.bar(x='Year', y=location,
                                    color='orange', label='El Niño',
                                    alpha=0.8, bar_width=5,
                                    width=1000, height=500).opts(axiswise=True)
    # Plot La Niña
    lanina_plot = df_lanina.hvplot.bar(x='Year', y=location,
                                    color='cadetblue', label='La Niña',
                                    alpha=0.8, bar_width=5,
                                    width=1000, height=500).opts(axiswise=True)
    # Overlay the plots
    combined_plot = (neutral_plot * elnino_plot * lanina_plot).opts(
        title=f'Observed Rx1day in {region_name}',
        xlabel='Year',
        ylabel='Precipitation (mm)',
        legend_position='top',
        tools=['hover', 'tap'],
        active_tools=['wheel_zoom'],
        toolbar='right',
        show_grid=False,
        axiswise=True,
    )

    return combined_plot

fig5_cebu_precip_plot = fig5_plot_rx1day(cebu_rx1day_neutral, cebu_rx1day_elnino, cebu_rx1day_lanina, 'Cebu', 'Cebu')
fig5_ncr_precip_plot = fig5_plot_rx1day(ncr_rx1day_neutral, ncr_rx1day_elnino, ncr_rx1day_lanina, 'NCR', 'Metro Manila')

# Temperature plot
def fig5_plot_temperature(df_tas, region_name, location):
    # Plot temperature
    temp_plot = df_tas.hvplot.line(x='Year', y=location,
                                color='red',
                                width=1000, height=500).opts(
                                    title=f'Observed TXx in {region_name}',
                                    xlabel='Year',
                                    ylabel='Temperature (°C)',
                                    tools=['hover', 'tap'],
                                    active_tools=['wheel_zoom'],
                                    toolbar='right',
                                    show_grid=False,
                                    axiswise=True,
                                )
    # Add points for each year
    points = df_tas.hvplot.scatter(x='Year', y=location, color='black', size=10)

    # Overlay the plots
    combined_plot = temp_plot * points 
    return combined_plot

fig5_cebu_temp_plot = fig5_plot_temperature(fig5_txx_cebu_df,'Cebu', 'Cebu')
fig5_ncr_temp_plot = fig5_plot_temperature(fig5_txx_mm_df,'NCR', 'Metro Manila')

# Figure 5 Layout
fig5_layout_cebu = pn.Row(fig4_cebu_map, pn.Column(fig5_cebu_precip_plot, fig5_cebu_temp_plot))
fig5_layout_ncr = pn.Row(fig4_ncr_map, pn.Column(fig5_ncr_precip_plot, fig5_ncr_temp_plot))

**Figure 8**

In [ ]:
# Read historical and ssp585 data 
fig8_philippines_historical = xr.open_dataset('data/fig8_philippines_historical.nc')
fig8_philippines_ssp585_mid = xr.open_dataset('data/fig8_philippines_mid_ssp585.nc')
fig8_philippines_ssp585_end = xr.open_dataset('data/fig8_philippines_end_ssp585.nc')
fig8_ncr_historical = xr.open_dataset('data/fig8_ncr_historical.nc')
fig8_ncr_ssp585_mid = xr.open_dataset('data/fig8_ncr_mid_ssp585.nc')
fig8_ncr_ssp585_end = xr.open_dataset('data/fig8_ncr_end_ssp585.nc')
fig8_cebu_historical = xr.open_dataset('data/fig8_cebu_historical.nc')
fig8_cebu_ssp585_mid = xr.open_dataset('data/fig8_cebu_mid_ssp585.nc')
fig8_cebu_ssp585_end = xr.open_dataset('data/fig8_cebu_end_ssp585.nc')
fig8_cdd_philippines_historical = xr.open_dataset('data/Fig8_Philippines_Historical_CDD.nc')
fig8_cdd_philippines_ssp585_mid = xr.open_dataset('data/Fig8_Philippines_mid_ssp585_CDD.nc')
fig8_cdd_philippines_ssp585_end = xr.open_dataset('data/Fig8_Philippines_end_ssp585_CDD.nc')
fig8_cdd_ncr_historical = xr.open_dataset('data/Fig8_NCR_Historical_CDD.nc')
fig8_cdd_ncr_ssp585_mid = xr.open_dataset('data/Fig8_NCR_mid_ssp585_CDD.nc')
fig8_cdd_ncr_ssp585_end = xr.open_dataset('data/Fig8_NCR_end_ssp585_CDD.nc')
fig8_cdd_cebu_historical = xr.open_dataset('data/Fig8_Cebu_Historical_CDD.nc')
fig8_cdd_cebu_ssp585_mid = xr.open_dataset('data/Fig8_Cebu_mid_ssp585_CDD.nc')
fig8_cdd_cebu_ssp585_end = xr.open_dataset('data/Fig8_Cebu_end_ssp585_CDD.nc')

# Read shapefile
fig8_philippines = gpd.GeoSeries(fig1_philippines['geometry'].union_all())
fig8_philippines = gpd.GeoDataFrame(geometry=fig8_philippines)

# Define colormaps
colormap_historical = cmocean.cm.rain
colormap_sequential = cm.broc_r
colormap_non_sequential = cm.navia_r
cdd_colormap_historical = cm.lajolla_r
cdd_colormap_sequential = cm.vik
cdd_colormap_period = cm.buda_r

# Function to create a historical plot
def historical_plot(data, variable, boundary, colormap, clabel):
    boundary_plot = boundary.hvplot(
        geo=True, 
        color=None,
        alpha=1,
        line_color='black',
        line_width=0.2,
        width=400,
        height=400)
    historical_plot = data[variable].hvplot(
        x='lon',
        y='lat',
        rasterize=True,
        geo=True,
        clabel=clabel,
        cmap=colormap,
        width=400,
        height=400,
        frame_width=400,
        frame_height=400
    ).opts(
        title='Historical Period (1995-2014)',
        xlabel='Longitude',
        ylabel='Latitude',
        xticks=5,
        yticks=2,
        tools=['hover','wheel_zoom'],
        toolbar='right',
        colorbar_position='bottom',
        axiswise=True
    )
    return historical_plot * boundary_plot

# Historical plot
fig8_philippines_historical_plot = historical_plot(fig8_philippines_historical, 'RX1day', fig8_philippines, colormap_historical, 'RX1day (mm/day)')
fig8_cebu_historical_plot = historical_plot(fig8_cebu_historical, 'RX1day', fig4_cebu, colormap_historical, 'RX1day (mm/day)')
fig8_ncr_historical_plot = historical_plot(fig8_ncr_historical, 'RX1day', fig4_ncr, colormap_historical, 'RX1day (mm/day)')

fig8_philippines_cdd_historical_plot = historical_plot(fig8_cdd_philippines_historical, 'CDD', fig8_philippines, cdd_colormap_historical, 'CDD (days)')
fig8_cebu_cdd_historical_plot = historical_plot(fig8_cdd_cebu_historical, 'CDD', fig4_cebu, cdd_colormap_historical, 'CDD (days)')
fig8_ncr_cdd_historical_plot = historical_plot(fig8_cdd_ncr_historical, 'CDD', fig4_ncr, cdd_colormap_historical, 'CDD (days)')

fig8_philippines_cdd_period_historical_plot = historical_plot(fig8_cdd_philippines_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig8_philippines, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')
fig8_cebu_cdd_period_historical_plot = historical_plot(fig8_cdd_cebu_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_cebu, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')
fig8_ncr_cdd_period_historical_plot = historical_plot(fig8_cdd_ncr_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_ncr, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')

# ssp585 plot
def ssp585_plot(data, variable, boundary, period, colormap):
    boundary_plot = boundary.hvplot(
        geo=True,
        color=None,
        alpha=1,
        line_color='black',
        line_width=0.2,
        width=400,
        height=400)
    ssp585_plot = data[variable].hvplot(
        x='lon',
        y='lat',
        rasterize=True,
        geo=True,
        clabel='Percentage Change (%)',
        cmap=colormap,
        width=400,
        height=400,
        frame_width=400,
        frame_height=400
    ).opts(
        title=f'SSP585 ({period})',
        xlabel='Longitude',
        ylabel='Latitude',
        xticks=5,
        yticks=2,
        tools=['hover','wheel_zoom'],
        toolbar='right',
        colorbar_position='bottom',
        axiswise=True
    )
    return ssp585_plot * boundary_plot

fig8_philippines_ssp585_mid_plot = ssp585_plot(fig8_philippines_ssp585_mid, 'RX1day', fig8_philippines, 'Mid-Century Period (2040-2059)', colormap_sequential)
fig8_cebu_ssp585_mid_plot = ssp585_plot(fig8_cebu_ssp585_mid, 'RX1day', fig4_cebu, 'Mid-Century Period (2040-2059)', colormap_sequential)
fig8_ncr_ssp585_mid_plot = ssp585_plot(fig8_ncr_ssp585_mid, 'RX1day', fig4_ncr, 'Mid-Century Period (2040-2059)', colormap_non_sequential)

fig8_philippines_ssp585_end_plot = ssp585_plot(fig8_philippines_ssp585_end, 'RX1day', fig8_philippines, 'End-Century Period (2080-2099)', colormap_sequential)
fig8_cebu_ssp585_end_plot = ssp585_plot(fig8_cebu_ssp585_end, 'RX1day', fig4_cebu, 'End-Century Period (2080-2099)', colormap_non_sequential)
fig8_ncr_ssp585_end_plot = ssp585_plot(fig8_ncr_ssp585_end, 'RX1day', fig4_ncr, 'End-Century Period (2080-2099)', colormap_non_sequential)

fig8_philippines_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_philippines_ssp585_mid, 'CDD', fig8_philippines, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_philippines_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_philippines_ssp585_end, 'CDD', fig8_philippines, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_ncr_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_ncr_ssp585_mid, 'CDD', fig4_ncr, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_ncr_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_ncr_ssp585_end, 'CDD', fig4_ncr, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_cebu_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_cebu_ssp585_mid, 'CDD', fig4_cebu, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_cebu_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_cebu_ssp585_end, 'CDD', fig4_cebu, 'End-Century Period (2080-2099)', cdd_colormap_sequential)

fig8_philippines_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_philippines_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig8_philippines, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_philippines_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_philippines_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig8_philippines, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_ncr_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_ncr_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_ncr, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_ncr_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_ncr_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_ncr, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_cebu_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_cebu_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_cebu, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_cebu_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_cebu_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_cebu, 'End-Century Period (2080-2099)', cdd_colormap_sequential)

# layout
def fig8_layout(historical_plot, mid, end):
    mid_layout = pn.Row(historical_plot, mid, width=1280)
    end_layout = pn.Row(historical_plot, end, width=1280)
    return mid_layout, end_layout

fig8_philippines_rx1day_mid, fig8_philippines_rx1day_end = fig8_layout(fig8_philippines_historical_plot, fig8_philippines_ssp585_mid_plot, fig8_philippines_ssp585_end_plot)
fig8_philippines_cdd_mid, fig8_philippines_cdd_end = fig8_layout(fig8_philippines_cdd_historical_plot, fig8_philippines_cdd_ssp585_mid_plot, fig8_philippines_cdd_ssp585_end_plot)
fig8_philippines_cdd_periods_mid, fig8_philippines_cdd_periods_end = fig8_layout(fig8_philippines_cdd_period_historical_plot, fig8_philippines_cdd_periods_ssp585_mid_plot, fig8_philippines_cdd_periods_ssp585_end_plot)
fig8_cebu_rx1day_mid, fig8_cebu_rx1day_end = fig8_layout(fig8_cebu_historical_plot, fig8_cebu_ssp585_mid_plot, fig8_cebu_ssp585_end_plot)
fig8_cebu_cdd_mid, fig8_cebu_cdd_end = fig8_layout(fig8_cebu_cdd_historical_plot, fig8_cebu_cdd_ssp585_mid_plot, fig8_cebu_cdd_ssp585_end_plot)
fig8_cebu_cdd_periods_mid, fig8_cebu_cdd_periods_end = fig8_layout(fig8_cebu_cdd_period_historical_plot, fig8_cebu_cdd_periods_ssp585_mid_plot, fig8_cebu_cdd_periods_ssp585_end_plot)
fig8_ncr_rx1day_mid, fig8_ncr_rx1day_end = fig8_layout(fig8_ncr_historical_plot, fig8_ncr_ssp585_mid_plot, fig8_ncr_ssp585_end_plot)
fig8_ncr_cdd_mid, fig8_ncr_cdd_end = fig8_layout(fig8_ncr_cdd_historical_plot, fig8_ncr_cdd_ssp585_mid_plot, fig8_ncr_cdd_ssp585_end_plot)
fig8_ncr_cdd_periods_mid, fig8_ncr_cdd_periods_end = fig8_layout(fig8_ncr_cdd_period_historical_plot, fig8_ncr_cdd_periods_ssp585_mid_plot, fig8_ncr_cdd_periods_ssp585_end_plot)

**Dashboard**

In [ ]:
# Create sidebar buttons
def create_button_sidebar(name,description):
    button = pn.widgets.Button(
        name=name,
        button_type="default",
        button_style='outline',
        description=description,
        styles={
            'box-shadow':'0px',
            "font-size": "20px",
            "background-color": "#FFF6E9", 
            "border": "2px solid black", 
            "padding": "3px",
            "width": '35%',
            'border-radius':'10px',
        }
    )
    return button

button_introduction = create_button_sidebar('Introduction', ' ')
button1 = create_button_sidebar('Figure 1',
                                'Map of Philippines (left) and Climatogram (right) of the four distinct climate regions of the Philippines.')
button2 = create_button_sidebar('Figure 2',
                                'Number of Landfalling and Non-Landfalling TCs in the Philippines (1951-2024)')
button4 = create_button_sidebar('Figure 4',
                                'Observed Annual Precipitation and Tas (1951-2020)')
button5 = create_button_sidebar('Figure 5',
                                'Observed Rx1Day and TXx in (1951-2020)')
button6 = create_button_sidebar('Figure 8',
                                'Percentage Change in Rx1Day and CDD compared to Historical Period (1995-2014)')
button_static = create_button_sidebar('Static Figures', 'Figures 1, 2, 4, 5, 8')
button_draft = create_button_sidebar('Draft Plots', 'View Draft Plots')

# Create figure 4 and 5 dropdowns
fig4_dropdown = pn.widgets.Select(
    name='Select Region',
    options=['National Capital Region (NCR)', 'Cebu'],
    value='National Capital Region (NCR)',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

fig5_dropdown = pn.widgets.Select(
    name='Select Region',
    options=['National Capital Region (NCR)', 'Cebu'],
    value='National Capital Region (NCR)',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

# Define hierarchical options for locations, variables, and time periods
options = {
    'Philippines': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    },
    'Cebu': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    },
    'National Capital Region (NCR)': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    }
}

# Create NestedSelect widget with three levels
fig8_nested_select = pn.widgets.NestedSelect(
    options=options,
    value={'Location': 'Philippines', 'Variable': 'Rx1day', 'Time Period': 'Mid-Century'},
    name='Select Region, Variable, and Time Period',
    levels=['Location', 'Variable', 'Time Period'],
    layout=pn.Row,
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

# Create widget to display static figures
static_dropdown = pn.widgets.Select(
    name='Select Figure',
    options=['Figure 1', 'Figure 2', 'Figure 4 (NCR, Cebu)', 
            'Figure 5 (NCR, Cebu)',
            'Figure 8 (Rx1day)', 'Figure 8 (CDD)', 'Figure 8 (CDD Periods)'],
    value='Figure 1',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

# Create widget to display draft plots
draft_dropdown = pn.widgets.Select(
    name='Select Figure',
    options=['Figure 1', 'Figure 2', 'Figure 4'],
    value='Figure 1',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

In [ ]:
# Static filepaths
fig1_png_path = 'static/fig1_combined.png'
fig2_png_path = 'static/Fig2_Number of TCs.png'
fig4_png_path_cebu = 'static/Fig4_Cebu.png'
fig4_png_path_ncr = 'static/Fig4_Manila.png'
fig5_png_path_ncr = 'static/Fig5_Manila.png'
fig5_png_path_cebu = 'static/Fig5_Cebu.png'
fig8_png_path_philippines_rx1day_mid = 'static/Fig8_Rx1Day_Philippines_Mid_Century.png'
fig8_png_path_philippines_rx1day_end = 'static/Fig8_Rx1Day_Philippines_End_Century.png'
fig8_png_path_phillippines_cdd_mid = 'static/Fig8_CDD_Philippines_Mid_Century.png'
fig8_png_path_phillippines_cdd_end = 'static/Fig8_CDD_Philippines_End_Century.png'
fig8_png_path_philippines_cdd_period_mid = 'static/Fig8_Periods_Philippines_Mid_Century.png'
fig8_png_path_philippines_cdd_period_end = 'static/Fig8_Periods_Philippines_End_Century.png'
fig8_png_path_ncr_rx1day_mid = 'static/Fig8_Rx1Day_NCR_Mid_Century.png'
fig8_png_path_ncr_rx1day_end = 'static/Fig8_Rx1Day_NCR_End_Century.png'
fig8_png_path_ncr_cdd_mid = 'static/Fig8_CDD_NCR_Mid_Century.png'
fig8_png_path_ncr_cdd_end = 'static/Fig8_CDD_NCR_End_Century.png'
fig8_png_path_ncr_cdd_period_mid = 'static/Fig8_Periods_NCR_Mid_Century.png'
fig8_png_path_ncr_cdd_period_end = 'static/Fig8_Periods_NCR_End_Century.png'
fig8_png_path_cebu_rx1day_mid = 'static/Fig8_Rx1Day_Cebu_Mid_Century.png'
fig8_png_path_cebu_rx1day_end = 'static/Fig8_Rx1Day_Cebu_End_Century.png'
fig8_png_path_cebu_cdd_mid = 'static/Fig8_CDD_Cebu_Mid_Century.png'
fig8_png_path_cebu_cdd_end = 'static/Fig8_CDD_Cebu_End_Century.png'
fig8_png_path_cebu_cdd_period_mid = 'static/Fig8_Periods_Cebu_Mid_Century.png'
fig8_png_path_cebu_cdd_period_end = 'static/Fig8_Periods_Cebu_End_Century.png'

# Descriptive caption images
fig1_caption = 'captions/Fig1_Caption.png'
fig1_info = 'captions/Fig1_info.png'
fig2_caption = 'captions/Fig2_info.png'
fig2_info = 'captions/Fig2_info_2.png'
fig4_mk = 'captions/MKtest.png'
fig4_caption = 'captions/Fig4_info.png'
fig4_info = 'captions/Fig4_info_2.png'
fig4_mk_info = 'captions/Fig4_info_3.png'
fig5_caption = 'captions/Fig5_caption.png'
fig8_caption = 'captions/Fig8_caption.png'
fig8_caption_2 = 'captions/Fig8_caption_2.png'
fig8_caption_3 = 'captions/Fig8_caption_3.png'
fig8_info = 'captions/Fig8_info.png'

def staticimage(image_path, caption_path, info_path):
    return pn.Column(
        pn.Row(
            pn.pane.PNG(caption_path, width=500),
            pn.pane.PNG(info_path, width=500)
        ),
        pn.pane.PNG(image_path, width=1280, height=720))

figure1_static = staticimage(fig1_png_path, fig1_caption, fig1_info)
figure2_static = staticimage(fig2_png_path, fig2_caption, fig2_info)
figure4_static = staticimage(fig4_png_path_ncr, fig4_caption, fig4_info)

def fig8_static(caption, info, ph1, ph2, c1, c2, n1, n2,):
    return pn.Column(
        pn.Row(
            pn.pane.PNG(caption, width=500),
            pn.pane.PNG(info, width=500)
        ),
        pn.Row(
            pn.pane.PNG(ph1, width=640, height=480),
            pn.pane.PNG(ph2, width=640, height=480)
        ),
        pn.Row(
            pn.pane.PNG(c1, width=640, height=480),
            pn.pane.PNG(c2, width=640, height=480)
        ),
        pn.Row(
            pn.pane.PNG(n1, width=640, height=480),
            pn.pane.PNG(n2, width=640, height=480)
        )
    )

fig8_rx1day_static = fig8_static(
    fig8_caption, fig8_info,
    fig8_png_path_philippines_rx1day_mid, fig8_png_path_philippines_rx1day_end,
    fig8_png_path_cebu_rx1day_mid, fig8_png_path_cebu_rx1day_end,
    fig8_png_path_ncr_rx1day_mid, fig8_png_path_ncr_rx1day_end,
)

fig8_cdd_static = fig8_static(
    fig8_caption_2, fig8_info,
    fig8_png_path_phillippines_cdd_mid, fig8_png_path_phillippines_cdd_end,
    fig8_png_path_cebu_cdd_mid, fig8_png_path_cebu_cdd_end,
    fig8_png_path_ncr_cdd_mid, fig8_png_path_ncr_cdd_end,
)

fig8_cdd_period_static = fig8_static(
    fig8_caption_3, fig8_info,
    fig8_png_path_philippines_cdd_period_mid, fig8_png_path_philippines_cdd_period_end,
    fig8_png_path_cebu_cdd_period_mid, fig8_png_path_cebu_cdd_period_end,
    fig8_png_path_ncr_cdd_period_mid, fig8_png_path_ncr_cdd_period_end,
)

fig1_draft_path = 'drafts/climate_map.png'
fig1_draft_path_climatogram = 'drafts/practice_climatogram.png'
fig1_draft_path_climatogram_2 = 'drafts/climatology_ver2.png'
fig1_draft_caption = 'captions/Fig1_draft_info.png'
fig1_draft_caption_2 = 'captions/Fig1_draft_info_2.png'
fig2_draft_path = 'drafts/TC_frequency.png'
fig2_draft_caption = 'captions/Fig2_draft_info.png'
fig4_draft_path = 'drafts/Fig4_Manila_2.png'
fig4_draft_caption = 'captions/Fig4_draft_info.png'

In [ ]:
def CreatePage1():
    # Add image for the introduction
    image_path='drafts/Coverslide.png'
    return pn.pane.PNG(image_path, width=1280, height=720)

def CreatePage2():
    return fig1_layout

def CreatePage3():
    return fig2_plot_with_labels

def CreatePage4():
    return pn.Column(
        fig4_dropdown,
        pn.bind(
            lambda value: 
                fig4_layout_cebu if value == 'Cebu' else
                fig4_layout_ncr if value == 'National Capital Region (NCR)' else
                pn.Row(pn.pane.Markdown("**Please select a valid region.**")),
            fig4_dropdown
        ),
        align='center'
    )

def CreatePage5():
    return pn.Column(
        fig5_dropdown,
        pn.bind(
            lambda value: 
                fig5_layout_cebu if value == 'Cebu' else
                fig5_layout_ncr if value == 'National Capital Region (NCR)' else
                pn.Row(pn.pane.Markdown("**Please select a valid region.**")),
            fig5_dropdown
        ),
        align='center'
    )

def CreatePage6():
    return pn.Column(
        fig8_nested_select,
        pn.bind(
            lambda value:
                fig8_philippines_rx1day_mid if value['Location'] == 'Philippines' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_philippines_rx1day_end if value['Location'] == 'Philippines' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_cebu_rx1day_mid if value['Location'] == 'Cebu' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_rx1day_end if value['Location'] == 'Cebu' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_ncr_rx1day_mid if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_ncr_rx1day_end if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_philippines_cdd_mid if value['Location'] == 'Philippines' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_philippines_cdd_end if value['Location'] == 'Philippines' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_cebu_cdd_mid if value['Location'] == 'Cebu' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_cdd_end if value['Location'] == 'Cebu' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_ncr_cdd_mid if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_ncr_cdd_end if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_philippines_cdd_periods_mid if value['Location'] == 'Philippines' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_philippines_cdd_periods_end if value['Location'] == 'Philippines' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                fig8_cebu_cdd_periods_mid if value['Location'] == 'Cebu' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_cdd_periods_end if value['Location'] == 'Cebu' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                fig8_ncr_cdd_periods_mid if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_ncr_cdd_periods_end if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                pn.Row(pn.pane.Markdown("**Please select a valid region, variable, and time period.**")),
            fig8_nested_select
        ),
        align='center'
    )

def CreatePage7():
    return pn.Column(static_dropdown,
        pn.bind(
            lambda value:
                figure1_static if value == 'Figure 1' else
                figure2_static if value == 'Figure 2' else
                pn.Column(
                    figure4_static,
                    pn.pane.PNG(fig4_png_path_cebu, width=1280, height=720),
                    pn.pane.PNG(fig4_mk_info),
                    pn.pane.PNG(fig4_mk, width=1080)
                ) if value == 'Figure 4 (NCR, Cebu)' else
                pn.Column(
                    pn.Row(
                        pn.pane.PNG(fig5_caption, width=500),
                        pn.pane.PNG(fig4_info, width=500)
                    ),
                    pn.pane.PNG(fig5_png_path_ncr, width=640, height=480),
                    pn.pane.PNG(fig5_png_path_cebu, width=640, height=480)
                ) if value == 'Figure 5 (NCR, Cebu)' else
                fig8_rx1day_static if value == 'Figure 8 (Rx1day)' else
                fig8_cdd_static if value == 'Figure 8 (CDD)' else
                fig8_cdd_period_static if value == 'Figure 8 (CDD Periods)' else
                None,
            static_dropdown),
        align='center'
    )


def CreatePage8():
    return pn.Column(draft_dropdown,
        pn.bind(
            lambda value:
                pn.Row(
                    pn.Column(
                        pn.pane.PNG(fig1_draft_caption, align='center'),
                        pn.Row(
                            pn.pane.PNG(fig1_draft_path, height=720, width=400),
                            pn.pane.PNG(fig1_draft_path_climatogram,height=720, width=400)
                        )),
                    pn.Column(
                        pn.pane.PNG(fig1_draft_caption_2, align='center'), 
                        pn.pane.PNG(fig1_draft_path_climatogram_2,height=720, width=640)
                    ),
                    ) if value == 'Figure 1' else
                pn.Column(
                    pn.pane.PNG(fig2_draft_caption, align='center'),
                    pn.pane.PNG(fig2_draft_path, height=720)
                ) if value == 'Figure 2' else
                pn.Column(
                    pn.pane.PNG(fig4_draft_caption, align='center'),
                    pn.pane.PNG(fig4_draft_path, width=1280, height=720)
                ) if value == 'Figure 4' else
                None,
            draft_dropdown),
        align='center'
    )

In [ ]:
mapping = {
    'Introduction': CreatePage1(),
    'Figure1': CreatePage2(),
    'Figure2': CreatePage3(),
    'Figure4': CreatePage4(),
    'Figure5': CreatePage5(),
    'Figure8': CreatePage6(),
    'StaticImages': CreatePage7(),
    'DraftImages': CreatePage8()
}

# Create sidebar layout
sidebar = pn.Column(
    pn.pane.Markdown('## Navigation', width=200),
    button_introduction,
    button1,
    button2,
    button4,
    button5,
    button6,
    button_static,
    button_draft,
    width=500,
    align='center'
)

# Create main area layout
main_area = pn.Column(
    mapping["Introduction"],
    width=800,
)

def show_page(page_key):
    main_area.clear()
    main_area.append(mapping[page_key])

button_introduction.on_click(lambda event: show_page("Introduction"))
button1.on_click(lambda event: show_page("Figure1"))
button2.on_click(lambda event: show_page("Figure2"))
button4.on_click(lambda event: show_page("Figure4"))
button5.on_click(lambda event: show_page("Figure5"))
button6.on_click(lambda event: show_page("Figure8"))
button_static.on_click(lambda event: show_page("StaticImages"))
button_draft.on_click(lambda event: show_page("DraftImages"))

Watcher(inst=Button(button_style='outline', description='View Draft Plots', name='Draft Plots', sizing_mode='stretch_width', styles={'box-shadow': '0px', ...}), cls=<class 'panel.widgets.button.Button'>, fn=<function <lambda> at 0x157984280>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [ ]:
template = pn.template.BootstrapTemplate(
    title=" Eileen's Internship Dashboard",
    sidebar=[sidebar],
    main=[main_area],
    header_background="#B35C3E", 
    sidebar_width=250,
)

pn.serve(template)

Launching server at http://localhost:54828
